# CAP 781
# Bagging, Random Forest

## Decision Trees as a high-variance model

* `Unpruned decision trees`, which are constructed by recursive partitioning and greedy search, can be `highly variable`. A small change in the training data could greatly distort the tree: the series of splits could be very different. In contrast, a procedure with **low variability** will yield **similar results** if applied
repeatedly to distinct data sets.

* Question: How can we reduce the variability of decision trees?

## Averaging 

* Suppose we have a random sample  $\{Z_1,\ldots, Z_n\}$
from some distribution such as:

$$
\{Z_i\}_{i=1}^{n} \stackrel{iid}{\sim} (\mu,\sigma^2),
$$
 
* with mean $E(Z_i)=\mu$ and variance $Var(Z_i)=\sigma^2$, for $i=1,\ldots,n$.

* Let $\bar{Z}=\frac{ \sum_{i=1}^{n}Z_i}{n}$ be the sample mean.
Since the observations are **independent** in a random sample, we have:

$$
E(\bar{Z})=\mu  \\
Var(\bar{Z}) = \frac{\sigma^2}{n}.
$$

* In words, the **variance of the average** is **less** than the variance of the sample elements, i.e., $Var(\bar{Z})\leq Var(Z_i)$.

## Model Averaging 

* Hence, a natural way to **reduce the variance** of a statistical learning method is to **take many training samples** (from the population of interest), build a separate prediction model using each training set, and **average the resulting predictions**.

* However, in practice, we have access to **only one training data**, **not many**. How can we solve this problem?

## Bootstrap Aggregation (Bagging) Algorithm


* At this point, we can apply `Bootstrap aggregation` or `bagging` which was proposed by Breiman (1996).
   * Let $\mathcal{D} = \{(\mathbf{x}_{1}, y_{1}), (\mathbf{x}_{2}, y_{2}), \ldots, (\mathbf{x}_{n}, y_{n})\}$ denote the **training data**.
   * Furthermore, consider a **learning algorithm** that takes the training data as its input and produces a prediction outcome such as $f(\mathcal{D})$.
   * We can draw a bootstrap sample by sampling from the training data **with replacement** such as: $\mathcal{D}^{*b} = \{(\mathbf{x}_{1}^{*b}, y_{1}^{*b}),   (\mathbf{x}_{2}^{*b}, y_{2}^{*b}), \ldots, (\mathbf{x}_{n}^{*b}, y_{n}^{*b})\}$ and repeat the procedure for $b = 1, \ldots,B$.


```
* An example of a bootstrap samples. Suppose this is your original dataset: [1,2,3,4]

   * A bootstrap sample drawn with replacement: [1,1,3,4]

   * A bootstrap sample drawn with replacement: [3,2,2,2]

   * A bootstrap sample drawn with replacement: [1,2,4,4]
```   
   
   * Then, we can apply the learning algorithm to the each bootstrap sample to learn the structure of the tree $f(\mathcal{D}^{*b})$, for $b = 1, \ldots,B$.

![](images/bootstrap.png)

   * For **classification** of a given a new observation ${x}_{n+1}$,  let $f(\mathcal{D}^{*b})$ be the predicted class of new observation by $bth$ learning algorithm. Then the
boostrap aggregated (bagged) prediction is given by:
   

   \begin{equation}
    \hat{y}^{bag}_{n+1}= \underset{(l \in \{1,\ldots,k,\ldots,K \})}{argmax} \sum_{b=1}^{B} I(f(\mathcal{D}^{*b})=c_l). \nonumber
   \end{equation}

   * Here, we record the class predicted by each of the algorithms and take a `majority vote`: the overall prediction is the **most commonly occurring class**
among the $B$ predictions.
* In Breiman (1996a) $B = 50$ was used in all numeric examples, but there is no particular reason for this choice. If the learning algorithm is
fast to compute then one can certainly use a larger B (say B = 100, 200, 300, 400, 500).

## Bagging: Classification Trees

* `Bagging` is a general-purpose procedure for **reducing the variability**, in other words, **improving instability** of a statistical learning method. However, `Bagging` is particularly useful for decision trees.

* To apply bagging to classification trees, we simply construct $B$ classification trees using $B$ bootstrapped training sets, apply the equation (3) for the prediction of new observations.

* These trees are grown  deep and are not pruned in general. Hence each individual tree has **high variance**. **Avereging** these $B$ trees **reduces the variance**.

![](images/bagging.png)

* The literature has shown that **bagging** demonstrated impressive **improvements in prediction accuracy** of decision trees by `ensembling` multiple trees into a single procedure.
* In machine learning world, `bagging` is also called as an `ensemble` learning method.

## Random Forests: Intuition

* Remember that if $\{Z_i\}_{i=1}^{n} \stackrel{idd}{\sim} (\mu,\sigma^2)$ is
a random sample from some distribution with mean $E(Z_i)=\mu$ and variance $Var(Z_i)=\sigma^2$, for $i=1,\ldots,n$. 
* Then, we have:

$$
E(\bar{Z})=\mu  \\
Var(\bar{Z}) = \frac{\sigma^2}{n}.
$$

* However, if the `pairwise correlation` between any two observations $Corr(Z_i,Z_j)=\rho$ exists, for $i \neq j$,
then:

$$
Var(\bar{Z}) = \rho \sigma^2 +  \frac{1-\rho}{n}\sigma^2.
$$

* In words, correlation between samples **limits the variance-reducing effect of averaging**.

* In a similar fashion, since each tree is built using a bootstrap sample from the original training
data, the trees in the **bagged ensemble will be somewhat correlated**.

* If we can **reduce correlation between the trees**, the trees will be more diverse and averaging can further improve the prediction.

## Random Forests: Definition

* Random forests is an ensemble learning classifier invented by Breiman (Breiman, 2001) and provide an **improvement over bagged trees** through `decorrelating` the trees.
*  Breiman(2001) aimed to reduce the correlations between the trees by using a **randomly selected** $m$ features from full set of $d$ features when building splits of decision tree, which can further enhance the variance reduction capability of bagging. (Note that random selection of features at the node level, not tree level).

![](images/rf.png)

* We typically choose $m = \sqrt{d}$--that is, the number of predictors considered at each split is approximately equal to the square root of the total number of features.
* If a random forest is built using $m = d$, then this amounts to simply bagging.
* Note that **Bagging** uses a **randomized method**, namely bootstrap, to generate a collection of classification trees. **Random forests** put another level of **randomization** on top of bagging by using a randomly selected a subset of predictors to construct each classification tree.

## Pros and Cons of Random Forests

Random forests remain a popular machine learning algorithm:

* They require **little data preparation** (no rescaling, handle continuous and discrete features, work well for classification and regression).
* They are one of the best performing off-the-shelf classifiers **without heavy tuning** of hyperparameters

Their main disadvantages are that:

* They are **not interpretable**.
* They are **slower** than decision trees because we are fitting multiple trees but can easily parallelize training because all trees are independent of each other.
* They require **more memory**.

## References

- James, G., Witten, D., Hastie, T., Tibshirani, R., James, G., Witten, D., and Tibshirani, R. (2021). Statistical learning. An introduction to statistical learning: with applications in R. https://www.statlearning.com/.
- Müller, A. C., and Guido, S. (2016). Introduction to machine learning with Python: A guide for data scientists. O'Reilly Media, Inc.
- Fan, J., Li, R., Zhang, C.H. and Zou, H., 2020. Statistical foundations of data science. CRC press.
- Wade, Corey, and Kevin Glynn. Hands-On Gradient Boosting with XGBoost and scikit-learn: Perform accessible machine learning and extreme gradient boosting with Python. Packt Publishing Ltd, 2020.
- Greenwell, B. M. (2022). Tree-based Methods for Statistical Learning in R. CRC Press.
- https://github.com/kuleshov/cornell-cs5785-2020-applied-ml/tree/main/notebooks
- https://ubc-cs.github.io/cpsc330/lectures/11_ensembles.html
- https://ubc-cs.github.io/cpsc330/lectures/12_feat-importances.html
- https://lewtun.github.io/hepml/lesson02_random-forest-deep-dive/
- https://inria.github.io/scikit-learn-mooc/trees/trees_module_intro.html
- https://www-users.cse.umn.edu/~kumar001/dmbook/slides/chap3_basic_classification.pdf
- https://www-users.cse.umn.edu/~kumar001/dmbook/slides/chap3_overfitting.pdf
- https://www-users.cse.umn.edu/~kumar001/dmbook/slides/chap4_ensemble.pdf
- https://stats.stackexchange.com/questions/357990/in-random-forest-why-is-a-random-subset-of-features-chosen-at-the-node-level-ra
- https://stats.stackexchange.com/questions/285006/random-forest-advantages-disadvantages-of-selecting-randomly-subset-features-fo

In [5]:
import session_info
session_info.show()